In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# importing keras layers and models --->model building libraries

Using TensorFlow backend.


In [2]:


# Initializing the CNN
classifier = Sequential()



#ADDING CNN LAYERS

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())



In [3]:
# Adding a fully connected layer---->Adding Dense Lyer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax')) # softmax for more than 2


In [4]:
# Compiling the CNN or configuring the model #optimaztion
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2


#importing Image Data Generator

from keras.preprocessing.image import ImageDataGenerator



In [5]:
#train and testing ..#IMAGEDATA GENERATOR FUNCTIONALITY TO TRAIN AND TEST SET
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

#TRAINING AND TEST THE MODEL

training_set = train_datagen.flow_from_directory('data/train',
                                                 target_size=(64, 64),
                                                 batch_size=5,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('data/test',
                                            target_size=(64, 64),
                                            batch_size=5,
                                            color_mode='grayscale',
                                            class_mode='categorical') 
classifier.fit_generator(
        training_set,
        steps_per_epoch=700, # No of images in training set
        epochs=10,
        validation_data=test_set,
        validation_steps=30)# No of images in test set


# SAVING THE MODEL
classifier.save('myclassifier.h5')
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
classifier.save_weights('model-bw.h5')

Found 660 images belonging to 6 classes.
Found 30 images belonging to 6 classes.

Epoch 1/10
700/700 [==============================] - 98s 140ms/step - loss: 0.6912 - accuracy: 0.7226 - val_loss: 1.5296 - val_accuracy: 0.9000
Epoch 2/10
700/700 [==============================] - 95s 136ms/step - loss: 0.2320 - accuracy: 0.9140 - val_loss: 9.3653e-05 - val_accuracy: 0.9333
Epoch 3/10
700/700 [==============================] - 99s 142ms/step - loss: 0.1169 - accuracy: 0.9600 - val_loss: 0.0324 - val_accuracy: 0.9667
Epoch 4/10
700/700 [==============================] - 99s 141ms/step - loss: 0.0787 - accuracy: 0.9757 - val_loss: 4.0317e-04 - val_accuracy: 0.9333
Epoch 5/10
700/700 [==============================] - 98s 139ms/step - loss: 0.0349 - accuracy: 0.9891 - val_loss: 2.3842e-06 - val_accuracy: 0.9333
Epoch 6/10
700/700 [==============================] - 97s 138ms/step - loss: 0.0413 - accuracy: 0.9891 - val_loss: 3.1523 - val_accuracy: 0.9667
Epoch 7/10
700/700 [================

In [38]:
classifier.save('myclassifier.h5')
import numpy as np
from keras.models import model_from_json
import operator
import cv2
import sys, os
from skimage.transform import resize

# Loading the model
json_file = open("model-bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
# load weights into new model
loaded_model.load_weights("model-bw.h5")
print("Loaded model from disk")
roi=cv2.imread("14.jpg")
print(roi)
#predicting output
def detect(frame):
    try:
        img = resize(frame,(64,64,1))
        img = np.expand_dims(img,axis=0)
        if(np.max(img)>1):
            img = img/255.0
        prediction = classifier.predict(img)
        prediction = classifier.predict_classes(img)
        return prediction
    except AttributeError:
        print("shape not found")
data=detect(roi)

Loaded model from disk
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


In [39]:
data

array([1], dtype=int64)